In [1]:
import pandas as pd
import rasterio
from rasterio import windows
from pyproj import Transformer

# Read the CSV file
csv_file = '\\Yehmh\\ZF\\seedtrapGPS_T4.csv'
points = pd.read_csv(csv_file)

# Define the transformer to convert TM2 coordinates to the TIFF CRS (if necessary)
# Replace 'EPSG:XXXX' with the correct EPSG code for your TIFF file's CRS
# transformer = Transformer.from_crs('EPSG:XXXX', 'EPSG:YYYY', always_xy=True)

# Open the TIFF file
# tif_file = 'h:\\Yehmh\\ZF\\202403_georeferenced\\202403_06_rgb.tif'
tif_file = '"F:\\ZhaoFeng_2024\\202403_RGBmosaic_georeferenced\\ZF03n2_clip.tif"'
with rasterio.open(tif_file) as src:
    for idx, row in points.iterrows():
        # Transform the coordinates
        tm2_x, tm2_y = row['tm2_x'], row['tm2_y']
        plot = row['Plot']
        
        # Calculate the pixel coordinates in the TIFF
        row, col = src.index(tm2_x, tm2_y)
        
        # Define the window to crop (40x40 meters)
        half_size = 20 / src.res[0]  # Assuming the resolution is in meters
        window = windows.Window(col - half_size, row - half_size, half_size * 2, half_size * 2)
        
        # Crop the image
        crop = src.read(window=window)
        
        # Save the cropped image
        crop_file = f'\\Yehmh\\ZF\\202403_georeferenced\\traps_images_T4\\{plot}.tif'
        profile = src.profile
        profile.update({
            'height': crop.shape[1],
            'width': crop.shape[2],
            'transform': windows.transform(window, src.transform)
        })
        
        with rasterio.open(crop_file, 'w', **profile) as dst:
            dst.write(crop)
        
        print(f'Cropped trap {plot} saved.')

print('Cropping completed.')


RasterioIOError: '"F:\ZhaoFeng_2024\202403_RGBmosaic_georeferenced\ZF03n2_clip.tif"' does not exist in the file system, and is not recognized as a supported dataset name.